In [1]:
from shared.simulators import ConstantEffectSimulator, ModeratedEffectSimulator, DataSimulator, EffectSimulatorCollection
from shared.analyzers import AnalyzerCollection, WLSAnalyzer, IPWAnalyzer, SIPWAnalyzer, MetaAnalyzer
from shared.result_processors import Results
from tqdm import trange
import matplotlib.pyplot as plt
import copy

In [2]:
N = 50
T = 100
alpha = 0.05
n_reps = 100
weight_type = "ATE"

# Effect simulators
effect_simulators = EffectSimulatorCollection()
# effect_simulators.add_effect_simulator(ConstantEffectSimulator("No Effect", N, 0., 0., 0.))
# effect_simulators.add_effect_simulator(ConstantEffectSimulator("Shared Effect", N, 1., 1., 0.))
# effect_simulators.add_effect_simulator(ConstantEffectSimulator("Distinct Effects", N, 1., 1., 0.3))
# effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Moderated Effects", N, 1., 1., 0.3, 1., 0.2))
effect_simulators.add_effect_simulator(ModeratedEffectSimulator("Noisy Moderated Effects", N, 1., 1., 0.3, 1., 0.2, epsilon_sd=0.5))

# Simulate effects once and fix
for effect_simulator in effect_simulators:
    effect_simulator.sample_params()

# Analyzers
analyzers = AnalyzerCollection()
wls_analyzer = WLSAnalyzer(name="WLS", alpha=alpha)
analyzers.add_analyzer(wls_analyzer)
analyzers.add_analyzer(MetaAnalyzer(wls_analyzer, name="Meta-WLS", alpha=alpha))

ipw_model_analyzer = IPWAnalyzer(name="IPW-Model", alpha=alpha, robust=False)
analyzers.add_analyzer(ipw_model_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_model_analyzer, name="Meta-IPW-Model", alpha=alpha))
ipw_robust_analyzer = IPWAnalyzer(name="IPW-Robust", alpha=alpha)
analyzers.add_analyzer(ipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_robust_analyzer, name="Meta-IPW-Robust", alpha=alpha))

sipw_robust_analyzer = SIPWAnalyzer(name="SIPW-Model", alpha=alpha, robust=False)
analyzers.add_analyzer(sipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_robust_analyzer, name="Meta-SIPW-Model", alpha=alpha))
sipw_robust_analyzer = SIPWAnalyzer(name="SIPW-Robust", alpha=alpha)
analyzers.add_analyzer(sipw_robust_analyzer)
analyzers.add_analyzer(MetaAnalyzer(sipw_robust_analyzer, name="Meta-SIPW-Robust", alpha=alpha))

ipw_model_analyzer = IPWAnalyzer(name="IPW-DR-Model", alpha=alpha, dr=True, robust=False)
analyzers.add_analyzer(ipw_model_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_model_analyzer, name="Meta-IPW-DR-Model", alpha=alpha))
ipw_dr_analyzer = IPWAnalyzer(name="IPW-DR-Robust", alpha=alpha, dr=True)
analyzers.add_analyzer(ipw_dr_analyzer)
analyzers.add_analyzer(MetaAnalyzer(ipw_dr_analyzer, name="Meta-IPW-DR-Robust", alpha=alpha))

# sipw_dr_model_analyzer = SIPWAnalyzer(name="SIPW-DR-Model", alpha=alpha, dr=True, robust=False)
# analyzers.add_analyzer(sipw_dr_model_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(sipw_dr_model_analyzer, name="Meta-SIPW-DR-Model", alpha=alpha))
# sipw_dr_robust_analyzer = SIPWAnalyzer(name="SIPW-DR-Robust", alpha=alpha, dr=True)
# analyzers.add_analyzer(sipw_dr_robust_analyzer)
# analyzers.add_analyzer(MetaAnalyzer(sipw_dr_robust_analyzer, name="Meta-SIPW-DR-Robust", alpha=alpha))

# Simulation loop
data_simulator = DataSimulator(N, T).simulate_history()
results = Results(data_simulator, effect_simulators, analyzers, n_reps=n_reps)
for rep_idx in trange(n_reps):
    for effect_simulator in effect_simulators:
        data_simulator = data_simulator.simulate_effects(effect_simulator)
        data_simulator = data_simulator.simulate_a()
        for analyzer in analyzers:          
            # Inferences for theta
            theta_inferences = analyzer.get_theta_inferences(data_simulator)
            results = results.add_theta_inferences(theta_inferences, data_simulator.theta.copy(), rep_idx, effect_simulator.i, analyzer.i)
            
            # Inferences for user estimates
            user_inferences = analyzer.get_user_inferences(data_simulator)
            results = results.add_user_inferences(user_inferences, data_simulator.user_effects.copy(), rep_idx, effect_simulator.i, analyzer.i)

100%|████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 38.56it/s]


In [3]:
digits = 3
results = results.process(digits=digits)
results.user_results_df.T

Coefficient                      Intercept                          \
Metric                            Avg Bias            Avg Sq. Bias   
Effect Simulator   Noisy Moderated Effects Noisy Moderated Effects   
WLS                          0.445 (0.004)                   0.208   
Meta-WLS                     0.435 (0.004)                   0.209   
IPW-Model                    0.001 (0.004)                   0.001   
Meta-IPW-Model               0.002 (0.004)                   0.022   
IPW-Robust                   0.001 (0.004)                   0.001   
Meta-IPW-Robust             -0.031 (0.004)                   0.057   
SIPW-Model                   0.012 (0.005)                   0.001   
Meta-SIPW-Model              0.008 (0.005)                   0.025   
SIPW-Robust                  0.012 (0.005)                   0.001   
Meta-SIPW-Robust             0.018 (0.005)                   0.025   
IPW-DR-Model                 0.009 (0.005)                   0.001   
Meta-IPW-DR-Model            0.007 (0.005)                   0.026   
IPW-DR-Robust                0.009 (0.005)                   0.001   
Meta-IPW-DR-Robust           0.019 (0.005)                   0.026   

Coefficient                                                         \
Metric                                 MSE                Coverage   
Effect Simulator   Noisy Moderated Effects Noisy Moderated Effects   
WLS                          0.278 (0.004)           0.681 (0.007)   
Meta-WLS                     0.237 (0.004)           0.521 (0.008)   
IPW-Model                    0.069 (0.001)           0.958 (0.003)   
Meta-IPW-Model               0.047 (0.001)           0.946 (0.003)   
IPW-Robust                   0.069 (0.001)           0.999 (0.001)   
Meta-IPW-Robust              0.066 (0.001)           0.966 (0.001)   
SIPW-Model                   0.095 (0.002)           0.959 (0.003)   
Meta-SIPW-Model              0.056 (0.001)           0.947 (0.003)   
SIPW-Robust                  0.095 (0.002)           0.951 (0.003)   
Meta-SIPW-Robust             0.057 (0.001)           0.943 (0.003)   
IPW-DR-Model                 0.104 (0.002)           0.954 (0.003)   
Meta-IPW-DR-Model            0.059 (0.001)           0.944 (0.003)   
IPW-DR-Robust                0.104 (0.002)           0.951 (0.003)   
Meta-IPW-DR-Robust           0.060 (0.001)           0.941 (0.003)   

Coefficient                                 
Metric                           CI Length  
Effect Simulator   Noisy Moderated Effects  
WLS                          1.150 (0.001)  
Meta-WLS                     0.884 (0.003)  
IPW-Model                    1.087 (0.001)  
Meta-IPW-Model               0.833 (0.002)  
IPW-Robust                   1.804 (0.002)  
Meta-IPW-Robust              1.019 (0.000)  
SIPW-Model                   1.256 (0.001)  
Meta-SIPW-Model              0.914 (0.004)  
SIPW-Robust                  1.250 (0.003)  
Meta-SIPW-Robust             0.913 (0.004)  
IPW-DR-Model                 1.288 (0.001)  
Meta-IPW-DR-Model            0.929 (0.004)  
IPW-DR-Robust                1.297 (0.003)  
Meta-IPW-DR-Robust           0.931 (0.005)